In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [8]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

In [9]:
class TabelParser:

    def parse_url(self,url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'lxml')
        return [(self.parse_table(table))\
                    for table in soup.find_all('table', class_="wikitable sortable")]
    def parse_table(self,table):
        n_rows = 0 
        n_columns = 0
        column_name = []
        
        for row in table.find_all('tr'):
            td_tags = row.find_all('td')
            
            if len(td_tags) > 0:
                n_rows += 1
                
                if n_columns == 0:
                    n_columns = len(td_tags)
                    
            th_tags = row.find_all('th')
            if len(th_tags) > 0 and len(column_name) == 0:
                for th in th_tags:
                    column_name.append(th.get_text())
        
        if len(column_name) > 0 and len(column_name) != n_columns:
            raise Exception("Headers do not match the number of columns")
            
        columns = column_name if len(column_name) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns=columns,index = range(0,n_rows))
        
        row_index = 0
        for row in table.find_all('tr'):
            column_index = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_index,column_index] = column.get_text()
                column_index += 1
            if len(columns) > 0:
                row_index += 1
                
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
            
        return df

In [10]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tp = TabelParser()
df = tp.parse_url(url)[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


In [11]:
a = df.copy()

In [12]:

a = a[a.Borough != 'Not assigned']

In [13]:

a.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


In [14]:
a = a.reset_index().drop(['index'],axis=1)
a = a.replace('\n','',regex=True)

In [15]:

a.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [16]:
a.rename(columns={'Neighbourhood\n':'Neighbourhood'},inplace=True)

In [17]:
a.Neighbourhood[a.Neighbourhood=='Not assigned '] = a.Borough

In [18]:
a.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [19]:
df_full = a.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ", ".join(x.astype(str))).reset_index()
df_full = df_full.sample(frac=1).reset_index(drop=True)
df_full.head(10)

,Postcode,Borough,Neighbourhood
0,M3N,North York,Downsview Northwest
1,M7A,Queen's Park,Not assigned
2,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
3,M1B,Scarborough,"Rouge, Malvern"
4,M1G,Scarborough,Woburn
5,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
6,M2K,North York,Bayview Village
7,M3K,North York,"CFB Toronto, Downsview East"
8,M5N,Central Toronto,Roselawn
9,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn"


In [20]:

df_full.shape

(103, 3)

In [21]:
url = 'http://cocl.us/Geospatial_data'
latlong = pd.read_csv(url)

In [22]:

latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
latlong.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [24]:
latlong.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
df_full = df_full.merge(latlong,on='Postcode')

In [26]:

df_full

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3N,North York,Downsview Northwest,43.761631,-79.520999
1,M7A,Queen's Park,Not assigned,43.662301,-79.389494
2,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724
3,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
4,M1G,Scarborough,Woburn,43.770992,-79.216917
5,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
6,M2K,North York,Bayview Village,43.786947,-79.385975
7,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763
8,M5N,Central Toronto,Roselawn,43.711695,-79.416936
9,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013
